In [5]:
### OUTER Baseline

import os
import csv

def create_evaluation_dataset(target_folder, actual_output_folder, output_csv, n=3):
    dataset = []

    # Get all target prompt .txt files in the target folder
    target_txt_files = [f for f in os.listdir(target_folder) if f.startswith("prompt_") and f.endswith('.txt')]

    for target_txt_file in target_txt_files:
        target_base_name = target_txt_file.replace("prompt_", "").replace(".txt", "")
        target_txt_path = os.path.join(target_folder, target_txt_file)

        # Find the corresponding expected output .py file
        expected_py_file = f"modified_{target_base_name}.py"
        expected_py_path = os.path.join(target_folder, expected_py_file)

        # Find the corresponding actual output .py file
        actual_output_file = f"prompt_{target_base_name}_generated_ICL.py"
        actual_output_path = os.path.join(actual_output_folder, actual_output_file)

        if not os.path.exists(expected_py_path):
            print(f"Expected output file not found for {target_txt_file}")
            continue

        if not os.path.exists(actual_output_path):
            print(f"Actual output file not found for {target_txt_file}")
            continue

        # Load the target prompt code
        with open(target_txt_path, 'r', encoding='utf-8') as f:
            target_prompt_code = f.read().strip()

        # Load the expected output code
        with open(expected_py_path, 'r', encoding='utf-8') as f:
            expected_output_code = f.read().strip()

        # Load the actual output code
        with open(actual_output_path, 'r', encoding='utf-8') as f:
            actual_output_code = f.read().strip()

        # Load in-context examples

        # Construct the input by concatenating in-context examples and the target prompt
        input_text = """
        You are an AI assistant that generates code for outer analyzers using the Mobileinsight-core Python library, a library that enables below-IP, \
        fine-grained mobile network analytics on end devices. It is a cross-platform package for mobile network monitoring and analysis.

        I will give the main target prompt that you need to follow in order to generate an outer analyzer.

        NOTE: PLEASE PROVIDE ONLY THE CODE AND NOTHING ELSE, AS THIS OUTPUT IS BEING DIRECTLY SAVED TO A .PY FILE AND RAN AUTONOMOUSLY. \
        ADDITIONALLY, ENSURE THAT YOU PROVIDE THE FULL COMPLETE CODE, AND DO NOT LEAVE OUT ANY PARTS FOR THE USER TO COMPLETE. THE CODE SHOULD FULLY RUN \
        WITH NO ADDITIONAL MODIFICATIONS REQUIRED.
        """
        input_text += f"Target Prompt:\n{target_prompt_code}\n"

        # Add the input-output pair to the dataset
        dataset.append({
            'input': input_text,
            'expected_output': expected_output_code,
            'actual_output': actual_output_code
        })

    # Write the dataset to a CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['input', 'expected_output', 'actual_output']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for data in dataset:
            writer.writerow(data)

    print(f"Evaluation dataset created and saved to {output_csv}")

# Example usage
target_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset'       # Folder with target prompts and expected outputs
output_csv = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\outer_dataset_basline.csv'             # Output CSV file path
actual_output_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_analyzers_baseline'

create_evaluation_dataset(target_folder, actual_output_folder, output_csv, n=3)


Evaluation dataset created and saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\outer_dataset_basline.csv


In [8]:
### INNER Baseline

import os
import csv

def create_evaluation_dataset(target_folder, actual_output_folder, output_csv, n=3):
    dataset = []

    # Get all target prompt .txt files in the target folder
    target_txt_files = [f for f in os.listdir(target_folder) if f.startswith("prompt_") and f.endswith('.txt')]

    for target_txt_file in target_txt_files:
        target_base_name = target_txt_file.replace("prompt_", "").replace(".txt", "")
        target_txt_path = os.path.join(target_folder, target_txt_file)

        # Find the corresponding expected output .py file
        expected_py_file = f"modified_{target_base_name}.py"
        expected_py_path = os.path.join(target_folder, expected_py_file)

        # Find the corresponding actual output .py file
        actual_output_file = f"prompt_{target_base_name}_baseline.py"
        actual_output_path = os.path.join(actual_output_folder, actual_output_file)

        if not os.path.exists(expected_py_path):
            print(f"Expected output file not found for {target_txt_file}")
            continue

        if not os.path.exists(actual_output_path):
            print(f"Actual output file not found for {target_txt_file}")
            continue

        # Load the target prompt code
        with open(target_txt_path, 'r', encoding='utf-8') as f:
            target_prompt_code = f.read().strip()

        # Load the expected output code
        with open(expected_py_path, 'r', encoding='utf-8') as f:
            expected_output_code = f.read().strip()

        # Load the actual output code
        with open(actual_output_path, 'r', encoding='utf-8') as f:
            actual_output_code = f.read().strip()

        # Load in-context examples

        # Construct the input by concatenating in-context examples and the target prompt
        input_text = """
        You are an AI assistant that generates code for inner analyzers using the Mobileinsight-core Python library, a library that enables below-IP, \
        fine-grained mobile network analytics on end devices. It is a cross-platform package for mobile network monitoring and analysis.

        I will give the main target prompt that you need to follow in order to generate an inner analyzer.

        NOTE: PLEASE PROVIDE ONLY THE CODE AND NOTHING ELSE, AS THIS OUTPUT IS BEING DIRECTLY SAVED TO A .PY FILE AND RAN AUTONOMOUSLY. \
        ADDITIONALLY, ENSURE THAT YOU PROVIDE THE FULL COMPLETE CODE, AND DO NOT LEAVE OUT ANY PARTS FOR THE USER TO COMPLETE. THE CODE SHOULD FULLY RUN \
        WITH NO ADDITIONAL MODIFICATIONS REQUIRED.
        """
        input_text += f"Target Prompt:\n{target_prompt_code}\n"

        # Add the input-output pair to the dataset
        dataset.append({
            'input': input_text,
            'expected_output': expected_output_code,
            'actual_output': actual_output_code
        })

    # Write the dataset to a CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['input', 'expected_output', 'actual_output']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for data in dataset:
            writer.writerow(data)

    print(f"Evaluation dataset created and saved to {output_csv}")

# Example usage
target_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset'       # Folder with target prompts and expected outputs
output_csv = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\inner_dataset_baseline.csv'             # Output CSV file path
actual_output_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_analyzers_baseline'

create_evaluation_dataset(target_folder, actual_output_folder, output_csv, n=3)


Evaluation dataset created and saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\inner_dataset_baseline.csv


In [1]:
### INNER ICL

import os
import csv
import random

def load_in_context_examples(example_folder, n=3):
    examples = []

    # Get lists of .txt and .py files
    txt_files = [f for f in os.listdir(example_folder) if f.startswith("prompt_") and f.endswith('.txt')]
    py_files = [f for f in os.listdir(example_folder) if f.startswith("modified_") and f.endswith('.py')]

    # Create a dictionary mapping base filenames to .py files
    py_file_dict = {f.replace("modified_", "").replace(".py", ""): f for f in py_files}

    for txt_file in txt_files:
        base_name = txt_file.replace("prompt_", "").replace(".txt", "")

        if base_name in py_file_dict:
            txt_file_path = os.path.join(example_folder, txt_file)
            py_file_path = os.path.join(example_folder, py_file_dict[base_name])

            # Load the prompt + code from the .txt file
            with open(txt_file_path, 'r', encoding='utf-8') as f:
                prompt_code = f.read().strip()

            # Load the expected code from the .py file
            with open(py_file_path, 'r', encoding='utf-8') as f:
                expected_code = f.read().strip()

            examples.append({
                'prompt_code': prompt_code,
                'expected_code': expected_code
            })

    # Randomly sample n examples
    return random.sample(examples, min(n, len(examples)))

def create_evaluation_dataset(example_folder, target_folder, actual_output_folder, output_csv, n=3):
    dataset = []

    # Get all target prompt .txt files in the target folder
    target_txt_files = [f for f in os.listdir(target_folder) if f.startswith("prompt_") and f.endswith('.txt')]

    for target_txt_file in target_txt_files:
        target_base_name = target_txt_file.replace("prompt_", "").replace(".txt", "")
        target_txt_path = os.path.join(target_folder, target_txt_file)

        # Find the corresponding expected output .py file
        expected_py_file = f"modified_{target_base_name}.py"
        expected_py_path = os.path.join(target_folder, expected_py_file)

        # Find the corresponding actual output .py file
        actual_output_file = f"prompt_{target_base_name}_generated_ICL.py"
        actual_output_path = os.path.join(actual_output_folder, actual_output_file)

        if not os.path.exists(expected_py_path):
            print(f"Expected output file not found for {target_txt_file}")
            continue

        if not os.path.exists(actual_output_path):
            print(f"Actual output file not found for {target_txt_file}")
            continue

        # Load the target prompt code
        with open(target_txt_path, 'r', encoding='utf-8') as f:
            target_prompt_code = f.read().strip()

        # Load the expected output code
        with open(expected_py_path, 'r', encoding='utf-8') as f:
            expected_output_code = f.read().strip()

        # Load the actual output code
        with open(actual_output_path, 'r', encoding='utf-8') as f:
            actual_output_code = f.read().strip()

        # Load in-context examples
        in_context_examples = load_in_context_examples(example_folder, n=n)

        # Construct the input by concatenating in-context examples and the target prompt
        input_text = """
        You are an AI assistant that generates code for inner analyzers using the Mobileinsight-core Python library, a library that enables below-IP, \
        fine-grained mobile network analytics on end devices. It is a cross-platform package for mobile network monitoring and analysis.

        For context, I will be giving a few examples of a prompt + outer analyzer code pairs along with their corresponding expected inner analyzer code.

        Then I will give the main target prompt that you need to follow in order to generate an inner analyzer.

        NOTE: PLEASE PROVIDE ONLY THE CODE AND NOTHING ELSE, AS THIS OUTPUT IS BEING DIRECTLY SAVED TO A .PY FILE AND RAN AUTONOMOUSLY. \
        ADDITIONALLY, ENSURE THAT YOU PROVIDE THE FULL COMPLETE CODE, AND DO NOT LEAVE OUT ANY PARTS FOR THE USER TO COMPLETE. THE CODE SHOULD FULLY RUN \
        WITH NO ADDITIONAL MODIFICATIONS REQUIRED.
        """

        for i, example in enumerate(in_context_examples):
            input_text += f"Example {i+1}:\n{example['prompt_code']}\n\n Expected Output:\n{example['expected_code']}\n\n"

        input_text += f"Target Prompt:\n{target_prompt_code}\n"

        # Add the input-output pair to the dataset
        dataset.append({
            'input': input_text,
            'expected_output': expected_output_code,
            'actual_output': actual_output_code
        })

    # Write the dataset to a CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['input', 'expected_output', 'actual_output']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for data in dataset:
            writer.writerow(data)

    print(f"Evaluation dataset created and saved to {output_csv}")

# Example usage
example_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset'  # Folder with in-context examples
target_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset'       # Folder with target prompts and expected outputs
output_csv = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\inner_dataset_ICL.csv'             # Output CSV file path
actual_output_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_analyzers_ICL'

create_evaluation_dataset(example_folder, target_folder, actual_output_folder, output_csv, n=3)


Evaluation dataset created and saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\inner_dataset_ICL.csv


In [4]:
### OUTER ICL

import os
import csv
import random

def load_in_context_examples(example_folder, n=3):
    examples = []

    # Get lists of .txt and .py files
    txt_files = [f for f in os.listdir(example_folder) if f.startswith("prompt_") and f.endswith('.txt')]
    py_files = [f for f in os.listdir(example_folder) if f.startswith("modified_") and f.endswith('.py')]

    # Create a dictionary mapping base filenames to .py files
    py_file_dict = {f.replace("modified_", "").replace(".py", ""): f for f in py_files}

    for txt_file in txt_files:
        base_name = txt_file.replace("prompt_", "").replace(".txt", "")

        if base_name in py_file_dict:
            txt_file_path = os.path.join(example_folder, txt_file)
            py_file_path = os.path.join(example_folder, py_file_dict[base_name])

            # Load the prompt + code from the .txt file
            with open(txt_file_path, 'r', encoding='utf-8') as f:
                prompt_code = f.read().strip()

            # Load the expected code from the .py file
            with open(py_file_path, 'r', encoding='utf-8') as f:
                expected_code = f.read().strip()

            examples.append({
                'prompt_code': prompt_code,
                'expected_code': expected_code
            })

    # Randomly sample n examples
    return random.sample(examples, min(n, len(examples)))

def create_evaluation_dataset(example_folder, target_folder, actual_output_folder, output_csv, n=3):
    dataset = []

    # Get all target prompt .txt files in the target folder
    target_txt_files = [f for f in os.listdir(target_folder) if f.startswith("prompt_") and f.endswith('.txt')]

    for target_txt_file in target_txt_files:
        target_base_name = target_txt_file.replace("prompt_", "").replace(".txt", "")
        target_txt_path = os.path.join(target_folder, target_txt_file)

        # Find the corresponding expected output .py file
        expected_py_file = f"modified_{target_base_name}.py"
        expected_py_path = os.path.join(target_folder, expected_py_file)

        # Find the corresponding actual output .py file
        actual_output_file = f"prompt_{target_base_name}_generated_ICL.py"
        actual_output_path = os.path.join(actual_output_folder, actual_output_file)

        if not os.path.exists(expected_py_path):
            print(f"Expected output file not found for {target_txt_file}")
            continue

        if not os.path.exists(actual_output_path):
            print(f"Actual output file not found for {target_txt_file}")
            continue

        # Load the target prompt code
        with open(target_txt_path, 'r', encoding='utf-8') as f:
            target_prompt_code = f.read().strip()

        # Load the expected output code
        with open(expected_py_path, 'r', encoding='utf-8') as f:
            expected_output_code = f.read().strip()

        # Load the actual output code
        with open(actual_output_path, 'r', encoding='utf-8') as f:
            actual_output_code = f.read().strip()

        # Load in-context examples
        in_context_examples = load_in_context_examples(example_folder, n=n)

        # Construct the input by concatenating in-context examples and the target prompt
        input_text = """
        You are an AI assistant that generates code for outer analyzers using the Mobileinsight-core Python library, a library that enables below-IP, \
        fine-grained mobile network analytics on end devices. It is a cross-platform package for mobile network monitoring and analysis.

        For context, I will be giving a few examples of a prompt + inner analyzer code pairs along with their corresponding expected outer analyzer code.

        Then I will give the main target prompt that you need to follow in order to generate an outer analyzer.

        NOTE: PLEASE PROVIDE ONLY THE CODE AND NOTHING ELSE, AS THIS OUTPUT IS BEING DIRECTLY SAVED TO A .PY FILE AND RAN AUTONOMOUSLY. \
        ADDITIONALLY, ENSURE THAT YOU PROVIDE THE FULL COMPLETE CODE, AND DO NOT LEAVE OUT ANY PARTS FOR THE USER TO COMPLETE. THE CODE SHOULD FULLY RUN \
        WITH NO ADDITIONAL MODIFICATIONS REQUIRED.
        """

        for i, example in enumerate(in_context_examples):
            input_text += f"Example {i+1}:\n{example['prompt_code']}\n\n Expected Output:\n{example['expected_code']}\n\n"

        input_text += f"Target Prompt:\n{target_prompt_code}\n"

        # Add the input-output pair to the dataset
        dataset.append({
            'input': input_text,
            'expected_output': expected_output_code,
            'actual_output': actual_output_code
        })

    # Write the dataset to a CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['input', 'expected_output', 'actual_output']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for data in dataset:
            writer.writerow(data)

    print(f"Evaluation dataset created and saved to {output_csv}")

# Example usage
example_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset'  # Folder with in-context examples
target_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset'       # Folder with target prompts and expected outputs
output_csv = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\outer_dataset_ICL.csv'             # Output CSV file path
actual_output_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_analyzers_ICL'

create_evaluation_dataset(example_folder, target_folder, actual_output_folder, output_csv, n=3)


Evaluation dataset created and saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\outer_dataset_ICL.csv


In [9]:
### INNER RAG
import os
import csv
import random
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Step 1: Load and chunk the codebase, then create embeddings
def index_codebase(directory):
    # Load all .py files from the directory
    loader = DirectoryLoader(
        path=directory,
        glob="**/*.py",  # Only .py files
        exclude=["**/__pycache__/**", "**/*.pyc"]
    )
    documents = loader.load()
    
    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separators=['\n\n', '\n', ' ', '']
    )
    chunks = text_splitter.split_documents(documents)
    
    # Create embeddings for each chunk
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(chunks, embeddings)
    
    return vectorstore


# Step 2: Load in-context examples
def load_in_context_examples(example_folder, n=3):
    examples = []

    # Get lists of .txt and .py files
    txt_files = [f for f in os.listdir(example_folder) if f.startswith("prompt_") and f.endswith('.txt')]
    py_files = [f for f in os.listdir(example_folder) if f.startswith("modified_") and f.endswith('.py')]

    # Create a dictionary mapping base filenames to .py files
    py_file_dict = {f.replace("modified_", "").replace(".py", ""): f for f in py_files}

    for txt_file in txt_files:
        base_name = txt_file.replace("prompt_", "").replace(".txt", "")

        if base_name in py_file_dict:
            txt_file_path = os.path.join(example_folder, txt_file)
            py_file_path = os.path.join(example_folder, py_file_dict[base_name])

            # Load the prompt + code from the .txt file
            with open(txt_file_path, 'r', encoding='utf-8') as f:
                prompt_code = f.read().strip()

            # Load the expected code from the .py file
            with open(py_file_path, 'r', encoding='utf-8') as f:
                expected_code = f.read().strip()

            examples.append({
                'prompt_code': prompt_code,
                'expected_code': expected_code
            })

    # Randomly sample n examples
    return random.sample(examples, min(n, len(examples)))


# Step 3: Retrieve relevant documents from the vector store
def retrieve_relevant_documents(vectorstore, prompt_inner_code):
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    relevant_docs = retriever.get_relevant_documents(prompt_inner_code)
    retrieved_context = "\n\n".join([doc.page_content for doc in relevant_docs])
    return retrieved_context


# Step 4: Create the evaluation dataset with retrieval context
def create_evaluation_dataset_with_rag(example_folder, target_folder, actual_output_folder, output_csv, vectorstore, n=3):
    dataset = []

    # Get all target prompt .txt files in the target folder
    target_txt_files = [f for f in os.listdir(target_folder) if f.startswith("prompt_") and f.endswith('.txt')]

    for target_txt_file in target_txt_files:
        target_base_name = target_txt_file.replace("prompt_", "").replace(".txt", "")
        target_txt_path = os.path.join(target_folder, target_txt_file)

        # Find the corresponding expected output .py file
        expected_py_file = f"modified_{target_base_name}.py"
        expected_py_path = os.path.join(target_folder, expected_py_file)

        # Find the corresponding actual output .py file
        actual_output_file = f"prompt_{target_base_name}_generated_RAG.py"
        actual_output_path = os.path.join(actual_output_folder, actual_output_file)

        if not os.path.exists(expected_py_path):
            print(f"Expected output file not found for {target_txt_file}")
            continue

        if not os.path.exists(actual_output_path):
            print(f"Actual output file not found for {target_txt_file}")
            continue

        # Load the target prompt code
        with open(target_txt_path, 'r', encoding='utf-8') as f:
            target_prompt_code = f.read().strip()

        # Load the expected output code
        with open(expected_py_path, 'r', encoding='utf-8') as f:
            expected_output_code = f.read().strip()

        # Load the actual output code
        with open(actual_output_path, 'r', encoding='utf-8') as f:
            actual_output_code = f.read().strip()

        # Retrieve relevant context using the vectorstore
        retrieved_context = retrieve_relevant_documents(vectorstore, target_prompt_code)

        # Load in-context examples
        in_context_examples = load_in_context_examples(example_folder, n=n)

        # Construct the input by concatenating in-context examples, retrieval context, and the target prompt
        input_text = """
        You are an AI assistant that generates code for outer analyzers using the Mobileinsight-core Python library, a library that enables below-IP, \
        fine-grained mobile network analytics on end devices. It is a cross-platform package for mobile network monitoring and analysis.

        For context, I will be giving a few examples of a prompt + inner analyzer code pairs along with their corresponding expected outer analyzer code.

        Then, I will give relevant context of the codebase from my RAG to also aid in generating the outer analyzer.

        Then I will give the main target prompt that you need to follow in order to generate an outer analyzer.

        NOTE: PLEASE PROVIDE ONLY THE CODE AND NOTHING ELSE, AS THIS OUTPUT IS BEING DIRECTLY SAVED TO A .PY FILE AND RAN AUTONOMOUSLY. \
        ADDITIONALLY, ENSURE THAT YOU PROVIDE THE FULL COMPLETE CODE, AND DO NOT LEAVE OUT ANY PARTS FOR THE USER TO COMPLETE. THE CODE SHOULD FULLY RUN \
        WITH NO ADDITIONAL MODIFICATIONS REQUIRED.
        """

        for i, example in enumerate(in_context_examples):
            input_text += f"Example {i+1}:\n{example['prompt_code']}\n\nExpected Output:\n{example['expected_code']}\n\n"

        input_text += f"Relevant Context from codebase:\n{retrieved_context}\n\nTarget Prompt:\n{target_prompt_code}\n"

        # Add the input-output pair to the dataset
        dataset.append({
            'input': input_text,
            'expected_output': expected_output_code,
            'actual_output': actual_output_code,
            'retrieval_context': retrieved_context

        })

    # Write the dataset to a CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['input', 'expected_output', 'actual_output', 'retrieval_context']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for data in dataset:
            writer.writerow(data)

    print(f"Evaluation dataset created and saved to {output_csv}")


# Example usage
example_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset'
target_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_working_dataset'
actual_output_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_outer_analyzers_RAG'
output_csv = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\outer_dataset_RAG.csv'

# Index the codebase and initialize the vectorstore
codebase_directory = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\mobile_insight'
vectorstore = index_codebase(codebase_directory)

# Create the evaluation dataset
create_evaluation_dataset_with_rag(example_folder, target_folder, actual_output_folder, output_csv, vectorstore, n=3)


C:\Users\bhull\AppData\Local\Temp\ipykernel_5920\3690642877.py:30: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
C:\Users\bhull\AppData\Local\Temp\ipykernel_5920\3690642877.py:74: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(prompt_inner_code)


Evaluation dataset created and saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\outer_dataset_RAG.csv


In [10]:
### INNER RAG
import os
import csv
import random
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Step 1: Load and chunk the codebase, then create embeddings
def index_codebase(directory):
    # Load all .py files from the directory
    loader = DirectoryLoader(
        path=directory,
        glob="**/*.py",  # Only .py files
        exclude=["**/__pycache__/**", "**/*.pyc"]
    )
    documents = loader.load()
    
    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separators=['\n\n', '\n', ' ', '']
    )
    chunks = text_splitter.split_documents(documents)
    
    # Create embeddings for each chunk
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(chunks, embeddings)
    
    return vectorstore


# Step 2: Load in-context examples
def load_in_context_examples(example_folder, n=3):
    examples = []

    # Get lists of .txt and .py files
    txt_files = [f for f in os.listdir(example_folder) if f.startswith("prompt_") and f.endswith('.txt')]
    py_files = [f for f in os.listdir(example_folder) if f.startswith("modified_") and f.endswith('.py')]

    # Create a dictionary mapping base filenames to .py files
    py_file_dict = {f.replace("modified_", "").replace(".py", ""): f for f in py_files}

    for txt_file in txt_files:
        base_name = txt_file.replace("prompt_", "").replace(".txt", "")

        if base_name in py_file_dict:
            txt_file_path = os.path.join(example_folder, txt_file)
            py_file_path = os.path.join(example_folder, py_file_dict[base_name])

            # Load the prompt + code from the .txt file
            with open(txt_file_path, 'r', encoding='utf-8') as f:
                prompt_code = f.read().strip()

            # Load the expected code from the .py file
            with open(py_file_path, 'r', encoding='utf-8') as f:
                expected_code = f.read().strip()

            examples.append({
                'prompt_code': prompt_code,
                'expected_code': expected_code
            })

    # Randomly sample n examples
    return random.sample(examples, min(n, len(examples)))


# Step 3: Retrieve relevant documents from the vector store
def retrieve_relevant_documents(vectorstore, prompt_inner_code):
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    relevant_docs = retriever.get_relevant_documents(prompt_inner_code)
    retrieved_context = "\n\n".join([doc.page_content for doc in relevant_docs])
    return retrieved_context


# Step 4: Create the evaluation dataset with retrieval context
def create_evaluation_dataset_with_rag(example_folder, target_folder, actual_output_folder, output_csv, vectorstore, n=3):
    dataset = []

    # Get all target prompt .txt files in the target folder
    target_txt_files = [f for f in os.listdir(target_folder) if f.startswith("prompt_") and f.endswith('.txt')]

    for target_txt_file in target_txt_files:
        target_base_name = target_txt_file.replace("prompt_", "").replace(".txt", "")
        target_txt_path = os.path.join(target_folder, target_txt_file)

        # Find the corresponding expected output .py file
        expected_py_file = f"modified_{target_base_name}.py"
        expected_py_path = os.path.join(target_folder, expected_py_file)

        # Find the corresponding actual output .py file
        actual_output_file = f"prompt_{target_base_name}_generated_RAG.py"
        actual_output_path = os.path.join(actual_output_folder, actual_output_file)

        if not os.path.exists(expected_py_path):
            print(f"Expected output file not found for {target_txt_file}")
            continue

        if not os.path.exists(actual_output_path):
            print(f"Actual output file not found for {target_txt_file}")
            continue

        # Load the target prompt code
        with open(target_txt_path, 'r', encoding='utf-8') as f:
            target_prompt_code = f.read().strip()

        # Load the expected output code
        with open(expected_py_path, 'r', encoding='utf-8') as f:
            expected_output_code = f.read().strip()

        # Load the actual output code
        with open(actual_output_path, 'r', encoding='utf-8') as f:
            actual_output_code = f.read().strip()

        # Retrieve relevant context using the vectorstore
        retrieved_context = retrieve_relevant_documents(vectorstore, target_prompt_code)

        # Load in-context examples
        in_context_examples = load_in_context_examples(example_folder, n=n)

        # Construct the input by concatenating in-context examples, retrieval context, and the target prompt
        input_text = """
        You are an AI assistant that generates code for inner analyzers using the Mobileinsight-core Python library, a library that enables below-IP, \
        fine-grained mobile network analytics on end devices. It is a cross-platform package for mobile network monitoring and analysis.

        For context, I will be giving a few examples of a prompt + outer analyzer code pairs along with their corresponding expected inner analyzer code.

        Then, I will give relevant context of the codebase from my RAG to also aid in generating the inner analyzer.

        Then I will give the main target prompt that you need to follow in order to generate an inner analyzer.

        NOTE: PLEASE PROVIDE ONLY THE CODE AND NOTHING ELSE, AS THIS OUTPUT IS BEING DIRECTLY SAVED TO A .PY FILE AND RAN AUTONOMOUSLY. \
        ADDITIONALLY, ENSURE THAT YOU PROVIDE THE FULL COMPLETE CODE, AND DO NOT LEAVE OUT ANY PARTS FOR THE USER TO COMPLETE. THE CODE SHOULD FULLY RUN \
        WITH NO ADDITIONAL MODIFICATIONS REQUIRED.
        """

        for i, example in enumerate(in_context_examples):
            input_text += f"Example {i+1}:\n{example['prompt_code']}\n\nExpected Output:\n{example['expected_code']}\n\n"

        input_text += f"Relevant Context from codebase:\n{retrieved_context}\n\nTarget Prompt:\n{target_prompt_code}\n"

        # Add the input-output pair to the dataset
        dataset.append({
            'input': input_text,
            'expected_output': expected_output_code,
            'actual_output': actual_output_code,
            'retrieval_context': retrieved_context

        })

    # Write the dataset to a CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['input', 'expected_output', 'actual_output', 'retrieval_context']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for data in dataset:
            writer.writerow(data)

    print(f"Evaluation dataset created and saved to {output_csv}")


# Example usage
example_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset'
target_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset'
actual_output_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_analyzers_RAG'
output_csv = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\inner_dataset_RAG.csv'

# Index the codebase and initialize the vectorstore
codebase_directory = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\mobile_insight'
vectorstore = index_codebase(codebase_directory)

# Create the evaluation dataset
create_evaluation_dataset_with_rag(example_folder, target_folder, actual_output_folder, output_csv, vectorstore, n=3)


Evaluation dataset created and saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\inner_dataset_RAG.csv
